In [1]:
from astroquery.gaia import Gaia

In [4]:
query = """
SELECT SOURCE_ID, RA, DEC, PARALLAX
FROM gaiadr3.gaia_source
"""

job = Gaia.launch_job(query)

results = job.get_results()

number_of_rows = len(results)
print(f"Number of rows: {number_of_rows}")


Number of rows: 2000


In [5]:
print(f"{'Source ID':<20} {'RA (deg)':<15} {'Dec (deg)':<15} {'Parallax (mas)':<15}")

# Print a separator
print('-' * 65)

# Print each row in the results
for row in results:
    print(f"{row['SOURCE_ID']:<20} {row['RA']:<15.6f} {row['DEC']:<15.6f} {row['PARALLAX']:<15.6f}")

Source ID            RA (deg)        Dec (deg)       Parallax (mas) 
-----------------------------------------------------------------
2048634533318668160  295.468799      36.454183       0.066850       
2048634533318668544  295.472632      36.454408       0.228584       
2048634533318668800  295.460640      36.454194       1.016095       
2048634533318669568  295.468344      36.455557       0.116721       
2048634533318671872  295.467607      36.459405       0.726312       
2048634533318672000  295.471171      36.459671       2.792498       
2048634533318673408  295.472207      36.462134       0.274945       
2048634533318674048  295.461279      36.462230       0.239683       
2048634533321319424  295.462879      36.452625       0.009994       
2048634563369026048  295.473663      36.463462       0.230889       
2048634563369037312  295.479520      36.464763       0.628589       
2048634563369051648  295.479592      36.467211       0.172893       
2048634563369064064  295.484170      

C:\Users\Horo\AppData\Local\Temp\ipykernel_16804\4198367839.py:8: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  print(f"{row['SOURCE_ID']:<20} {row['RA']:<15.6f} {row['DEC']:<15.6f} {row['PARALLAX']:<15.6f}")


In [7]:
# Extract RA and Dec
coordinates = results[['RA', 'DEC']]

# Prepare a list of coordinates as a string for Aladin Lite
coords_str = ', '.join([f"{row['RA']},{row['DEC']}" for row in coordinates])


295.468799309167,36.454182669357124, 295.47263208756914,36.454408393904856, 295.4606401411724,36.45419403369547, 295.4683444909502,36.45555690404011, 295.46760669153167,36.45940502547969, 295.47117097046805,36.459670652034085, 295.47220741266773,36.462134334786455, 295.46127850902394,36.462230272437104, 295.4628785119084,36.45262546156146, 295.47366313996685,36.46346237372367, 295.47951951156,36.46476278915705, 295.4795918960509,36.46721066977801, 295.48416989342365,36.470072405571976, 295.4858792129008,36.47159444466038, 296.18380739840586,37.57026452206905, 296.18098730327955,37.57191967561929, 296.18121809988577,37.574273414369415, 296.18455020166755,37.568220699899626, 296.19483335738977,37.57209775708631, 296.19840902915786,37.5778491975814, 296.19293929325113,37.579106131682245, 296.190358459921,37.579865931672416, 296.1998946645881,37.58208606752793, 296.19310033665494,37.587306223785276, 296.19466309318426,37.57756348988103, 296.1927565077716,37.58159929918511, 296.191323569222